## Rename Oddity Folders and Files(.sh)

This script should work on both session folders pulled from the drive as well as from the raw folder. (e.g BAY11107_20231128 or 11107_20231128). 

This does not look for specific file extensions since the file basename are the same. This means that any edat, edat2, and txt will all be renamed to the same format. 

In [ ]:
#!/bin/bash

renamed_folders_count=0

rename_session_folder_and_files() {
    local id_folder="$1"
    local session_folder="$2"
    local id="$3"
    local date="$4"

    local already_renamed=true
    for file in "${session_folder}"/*; do
        if [[ -f "$file" && ! $file =~ sub-${id}_date-${date}_proj-oddity_response ]]; then
            already_renamed=false
            break
        fi
    done

    if $already_renamed; then
        return
    fi

    local new_session_folder="${id_folder}${date}"

    if [[ "$session_folder" != "$new_session_folder" ]]; then
        mv "$session_folder" "$new_session_folder"
        echo "$session_folder -> $new_session_folder"
        ((renamed_folders_count++))
    fi

    for file in "${new_session_folder}"/*; do
        if [[ -f "$file" ]]; then
            local extension="${file##*.}"
            local new_file="${new_session_folder}/sub-${id}_date-${date}_proj-oddity_response.${extension}"
            mv "$file" "$new_file"
        fi
    done
}

for id_folder in BAY*/ AVP*/; do
    if [[ -d "$id_folder" ]]; then
        id=${id_folder%/}  # Remove trailing slash

        for session_folder in "${id_folder}"*/; do
            if [[ -d "$session_folder" ]]; then
                session_name=$(basename "$session_folder")
                session_date="${session_name##*[-_]}"
                rename_session_folder_and_files "$id_folder" "$session_folder" "$id" "$session_date"
            fi
        done
    fi
done

if [ $renamed_folders_count -eq 0 ]; then
    echo "No folders were renamed."
else
    echo "Total session folders renamed: $renamed_folders_count"
fi
